<a href="https://colab.research.google.com/github/zuhayerror3i8/Deep-Learning-A-Z---2025---Neural-Networks-AI-ChatGPT-Prize/blob/main/Part%2004%20-%20Self%20Organizing%20Maps%20(SOM)/mega_case_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mega Case Study

## Part 1 - SOM

### Importing the libraries

In [ ]:
!pip install minisom

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Importing the dataset

In [ ]:
dataset = pd.read_csv('Credit_Card_Applications.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

### Feature Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0, 1))
X = sc.fit_transform(X)

### Training the SOM

In [ ]:
from minisom import MiniSom
som = MiniSom(x=10, y=10, input_len=15, sigma=1.0, learning_rate=0.5)
som.random_weights_init(X)
som.train_random(data=X, num_iteration=100)

### Visualizing the results

In [ ]:
from pylab import bone, pcolor, colorbar, plot, show
bone()
pcolor(som.distance_map().T)
colorbar()
markers = ['o', 's']
colors = ['r', 'g']
for i, x in enumerate(X):
    w = som.winner(x)
    plot(w[0] + 0.5,
         w[1] + 0.5,
         markers[y[i]],
         markeredgecolor=colors[y[i]],
         markerfacecolor='None',
         markersize=10,
         markeredgewidth=2)
show()

### Finding the frauds

In [ ]:
mappings = som.win_map(X)
frauds = np.concatenate((mappings[(6, 3)], mappings[(5, 4)]), axis=0)
frauds = sc.inverse_transform(frauds)

### Printing the fraudulent clients

In [ ]:
print('Fraud Customer IDs')
for i in frauds[:, 0]:
  print(int(i))

## Part 2 - Going from Unsupervised to Supervised Deep Learning

### Creating the Matrix of Features

In [ ]:
customers = dataset.iloc[:, 1:].values

### Creating the Dependent Variable

In [ ]:
is_fraud = np.zeros(len(dataset))
for i in range(len(dataset)):
  if dataset.iloc[i, 0] in frauds:
    is_fraud[i] = 1

## Part 3 - ANN

### Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
customers = sc.fit_transform(customers)

### Building the ANN

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=2, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the ANN

In [ ]:
ann.fit(customers, is_fraud, batch_size=1, epochs=10)

### Predicting the test set results

In [ ]:
y_pred = ann.predict(customers)
y_pred = np.concatenate((dataset.iloc[:, 0:1].values, y_pred), axis=1)
y_pred = y_pred[y_pred[:, 1].argsort()]

In [ ]:
print(y_pred)